In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import copy
from tqdm import tqdm
import re

import pyarrow as pa
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings(action='ignore')
pd.set_option("display.max_columns", None)

In [2]:
oct_2019_parquet_path = os.path.join("data", "parquet_data", "oct_2019.parquet")
nov_2019_parquet_path = os.path.join("data", "parquet_data", "nov_2019.parquet")

In [3]:
oct_2019_parquet_df = pd.read_parquet(oct_2019_parquet_path, engine="pyarrow")

In [4]:
nov_2019_parquet_df = pd.read_parquet(nov_2019_parquet_path, engine="pyarrow")

In [5]:
oct_2019_parquet_df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_time_ymd,event_time_hms
0,2019-10-01 00:00:00+00:00,view,44600062,2103807459595387724,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,00:00:00
1,2019-10-01 00:00:00+00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2019-10-01,00:00:00
2,2019-10-01 00:00:01+00:00,view,17200506,2053013559792632471,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,2019-10-01,00:00:01
3,2019-10-01 00:00:01+00:00,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2019-10-01,00:00:01
4,2019-10-01 00:00:04+00:00,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2019-10-01,00:00:04
...,...,...,...,...,...,...,...,...,...,...,...
42448759,2019-10-31 23:59:58+00:00,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193,2019-10-31,23:59:58
42448760,2019-10-31 23:59:58+00:00,view,10800172,2053013554994348409,None,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8,2019-10-31,23:59:58
42448761,2019-10-31 23:59:58+00:00,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd,2019-10-31,23:59:58
42448762,2019-10-31 23:59:59+00:00,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204,2019-10-31,23:59:59
